#1 Чем плохи деструкторы в питоне?

Касательно деструкторов:

  1. Деструктор будет работать не корректно в случае круговой ссылки. 
Что это значит. Это значит, что один класс ссылается на другой, котороый ссылается на первый
(пример ниже)

In [1]:
class Vehicle():
  def __init__(self, id:int, car):
    self.id = id
    self.manufacturer = car
    print(f'Vehicle with id: {self.id} by {self.manufacturer} created!')

  def __del__(self):
    print(f'Vehicle with id: {self.id} by {self.manufacturer} destroyed!')

class Car():
  def __init__(self, id):
    self.id = id
    self.manufacturer = Vehicle(id, self)
    print(f'Car with id: {self.id} by {self.manufacturer} created!')

  def __del__(self):
    print(f'Car with id: {self.id} by {self.manufacturer} destroyed!')

In [ ]:
polo = Car(1)
del polo
#Как можно видеть, экземпляры создались, но не были удалены
#И при этом деструктор не был вызван

Vehicle with id: 1 by <__main__.Car object at 0x7f76ca3a52b0> created!
Car with id: 1 by <__main__.Vehicle object at 0x7f76ca3a5550> created!


  2. Деструктор так же не будет работать в том случае, если возникает вызов исключения про сборке экземпляра (в init). То есть, если в конструкторе вызывается обработчик ошибок, то будет вызван метод деструктора. Но на самом деле объект создан не будет (пример ниже)

In [ ]:
class Car():
  def __init__(self, speed):
    if speed > 320:
      raise Exception('Too mach too young too fast!')
    else:
      print('New car created')

  def __del__(self):
    print('Cleaning resources')

In [ ]:
octavia = Car(230)# Тут все прошло нормально, так как 230 < 320
gt_911 = Car(350)

#Ну и да. Оно упало. А, и 

New car created


Exception: ignored

#2. Почему может быть плох import в начале файла?

Вспоминая, что в python существует такая штука как PEP8, согласно которой импорты нужно располагать в начале файла. Чем может быть плох import в начале файла...ну на ум риходит пока что только условные импорты.
К примеру, если у нас есть причниы, по которой одна из версий модуля не работает в определенной операционной системе, или версии самого python. Так вот, можно выполнять импорты в зависимости от выполнения условий (пример ниже)

In [ ]:
#Пример простой, но для демонстрации сойдет.

purpose = str(input())

purpose_list = ['visualisation', 'calculating']

if purpose in purpose_list:
  if purpose == purpose_list[0]:
    import pandas as module
    print('Импортирован модуль визуализации')
  if purpose == purpose_list[1]:
    import numpy as module
    print('Импортирован модуль для вычислений')


visualisation
Импортирован модуль визуализации


In [ ]:
%pip freeze -> requriments.txt

In [ ]:
with open('requriments.txt', 'r', encoding='UTF-8')as f:
  f = f.read()
  if purpose == purpose_list[0]:
    print('pandas' in f)
  if purpose == purpose_list[1]:
    print('numpy' in f)
#Вывод будет true в любом случае, если использовать google colab, так как в нем все модули уже установлены...

True


#3. Как в питоне «вывалиться» из любой вложенности или рекурсии по аналогу «goto:»?

В питоне нет привычного или прямого аналога goto:, но есть условные операторы, такие как if, elif, else. Также есть методы break и continue. If else - понятно, если что-то то делаем что-то, если нет, то иначе. elif - это альтернатива if. То есть, если if условие не выполнилось, будет проверяться условие elif'а. 


С break и continue все тоже блоее менее ясно из названия. Первый прервет выполнение цикла или функции, вотрой продолжит, если условие выполнилось.

Также можно использовать механизм флагов. Объямить преременную, в которой задать определенное значение (чаще всего True или False), а затем добавить к условию проверку флага. Как только необходимое условие будет выполнено, то заменяем значение флага на противоположное или на то, которе не пройдет условие проверки.

In [ ]:
def factorial_recursive(n):
    if n == 1:
        return n
    elif n > 10:
        return n*factorial_recursive(n-1)
    else: return n + 1

factorial_recursive(8)

9

#4.  Сервер отдает сообщение один раз (1кб, бинарь). Его необходимо сохранить в базе, нельзя потерять(!). Как бы вы решали такую задачу?

Если я парвильно понял, то задача состоит в том, чтобы удостовериться, что сообщение сохранено. Во всяком случае бубу отталкиваться от этого

Начнем не совсем с очевидного. В первую очередь создадим две переменные тип str с пустой строкой(''). Далее запишем сообщение в одну из переменных и проверим, совпадают ли они. Если нет, то передадим сообщение и этой переменной, параллельно запишем время и дату системы. Далее нам нужно записать переменную в файл с расширением .bin (это же бинарник). В качестве названия передадим следующее f'Server_resp {переменная для храрнения времени}.bin'. Так можно будет отселдить время получения сообщения. Далее нам нужно будет передать это сообщение в бд. НО предварительно убедимся, что записали файл правильно. Для этого прочитаем его содержимое и сверим со значением переменной, в которой оно хранилось. Если они схожи, значит в случае чего обратимся к файлу. У нас есть два пути:

1. Проверить, доступна ли бд в данный момент времени или нет. Если да, то записать в нее сообщение, а затем проверить появилось ли оно в базе. Если нет, то проверять доступнось и если база доступна, то записываем еще раз и так покругу, пока не запишеться. (В случае перезапуска программы у нас остается файл, так что можем подтянуть сообщение оттуда.)

2. Использовать ORM SQLAlchmy.Session. В чем суть. Это транзакция, которая открывается только перед записью в бд, а значит держать её открытой все это время не нужно. Также используя сессии гарантируется, что объекты, загруженные в базу данных будут присутствовать в одном экземпляре. И не стоит забывать, что использование сессий в некоторых случаях может ускорить выполнение запросов. +, тут не уверен, так как сессиями пользовался мало, использование данного инструмента гарантирует запись в бд, если она доступна